In [5]:
import pandas as pd
from project_config import PREPROCESSED_2012_DATA_FILE, PREPROCESSED_2022_DATA_FILE
from utils.file_encoding_detector import detect_encoding
from utils.read_csv_file import read_csv_file

file_2012 = PREPROCESSED_2012_DATA_FILE
file_2022 = PREPROCESSED_2022_DATA_FILE

df_1 = read_csv_file(file_2012)
df_2 = read_csv_file(file_2022)

df_2.group.nunique()

673

In [ ]:
from project_config import YEARBOOK_2012_DATA_DIR


source_data_dir = YEARBOOK_2012_DATA_DIR

files_to_check_against = list(source_data_dir.glob("*.xls*"))

print(files_to_check_against)

In [2]:
from project_config import TRAINING_DATA_FILE, TRAINING_INFERENCE_DATA_FILE
from utils.read_csv_file import read_csv_file

df1 = read_csv_file(TRAINING_DATA_FILE)
df2 = read_csv_file(TRAINING_INFERENCE_DATA_FILE)

print(len(df1), len(df2))

17403 98742


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.models import KeyedVectors

# Load pre-trained Word2Vec model (Google News dataset)
word_vectors = KeyedVectors.load_word2vec_format(
    "GoogleNews-vectors-negative300.bin", binary=True
)

# Words to visualize
words = ["king", "queen", "man", "woman"]
vectors = [word_vectors[word] for word in words]

# Reduce to 2D for visualization
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(vectors)

# Plot
plt.figure(figsize=(8, 8))
for i, word in enumerate(words):
    plt.scatter(reduced_vectors[i, 0], reduced_vectors[i, 1], label=word)
    plt.text(reduced_vectors[i, 0] + 0.02, reduced_vectors[i, 1], word, fontsize=12)
plt.title("Word Embedding Visualization (King, Queen, Man, Woman)")
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
from project_config import TRAINING_DATA_FILE
from utils.read_csv_file import read_csv_file

df = read_csv_file(TRAINING_DATA_FILE)

,text,yearbook_source,section,group,row_id,is_title,is_empty,original_index,label
0,1-1 Divisions of Administrative Areas in Chin...,2012,General Survey,B0101e,1,yes,no,0,title
1,"EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...",2012,General Survey,B0101e,2,no,yes,1,empty
2,"(unit), EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMP...",2012,General Survey,B0101e,3,no,no,2,metadata
3,"EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...",2012,General Survey,B0101e,4,no,yes,3,empty
4,"Provinces Autonomous , Number of, Cities, Num...",2012,General Survey,B0101e,5,no,no,4,header
...,...,...,...,...,...,...,...,...,...
17398,"United Kingdom, 0.66, 0.65, 0.78, 0.75, 0.78,...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,47,no,no,17398,table_data
17399,"Australia, 1.73, 1.09, 1.31, 1.34, 1.44, 1.45...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,48,no,no,17399,table_data
17400,"New Zealand, 2.2, 1.39, 1.41, 1.45, 1.52, 1.5...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,49,no,no,17400,table_data
17401,"EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...",2022,Appendix II. A Comparison of Indicators of Eco...,E29-15,50,no,yes,17401,empty


# Inspect Final Output Files

In [15]:
import pandas as pd
from utils.read_csv_file import read_csv_file

from project_config import (
    INFERENCE_INPUT_DATA_FILE,
    RAW_INFERENCE_OUTPUT_DATA_FILE,
    CLEANED_INFERENCE_OUTPUT_DATA_FILE,
    TRAINING_DATA_FILE,
    CLEANED_TRAINING_OUTPUT_DATA_FILE,
    COMBINED_CLEANED_OUTPUT_DATA_FILE,
)

df = pd.read_csv(CLEANED_INFERENCE_OUTPUT_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"inference file blank text cells: {blank_cells}")

df = read_csv_file(TRAINING_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"original_training file blank text cells: {blank_cells}")

df = pd.read_csv(CLEANED_TRAINING_OUTPUT_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"cleaned_training file blank text cells: {blank_cells}")

df = pd.read_csv(COMBINED_CLEANED_OUTPUT_DATA_FILE)
blank_cells = df["text"].isnull().sum()
print(f"combined file blank text cells: {blank_cells}")

inference file blank text cells: 0


root - INFO - Detected encoding: ISO-8859-1 with confidence 0.73
root - INFO - File read successfully from C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv


original_training file blank text cells: 0
cleaned_training file blank text cells: 93
combined file blank text cells: 93


In [14]:
import logging
from pathlib import Path
from data_processing.post_process_inference_data import (
    process_inference_training_results,
)
from project_config import TRAINING_DATA_FILE, CLEANED_TRAINING_OUTPUT_DATA_FILE

# Optional: Setup logging if you want to see output
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def clean_training_data():
    """
    Re-clean the training input data using the same cleaning pipeline
    used in the inference pipeline.
    """
    input_file = Path(TRAINING_DATA_FILE)
    output_file = Path(CLEANED_TRAINING_OUTPUT_DATA_FILE)

    logging.info(f"Cleaning training data from: {input_file}")
    process_inference_training_results(
        input_file_path=input_file, output_file_path=output_file
    )
    logging.info(f"Cleaned training data saved to: {output_file}")


if __name__ == "__main__":
    clean_training_data()

root - INFO - Cleaning training data from: C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv
data_processing.post_process_inference_data - INFO - Start processing file (C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv)...


[DEBUG] Retrieved username: xzhan
[DEBUG] Log file path: C:\github\table_lense\logs\xzhan_20250322_172204.log
[DEBUG] Logging successfully configured with a single file handler


data_processing.post_process_inference_data - INFO - Detected encoding: ISO-8859-1 with confidence 0.73
data_processing.post_process_inference_data - INFO - File read successfully from C:\github\table_lense\pipeline_data\training_inference\training\training_data_v1.csv
data_processing.post_process_inference_data - INFO - DataFrame loaded:
                                                text  yearbook_source  \
0  1-1  Divisions of Administrative Areas in Chin...             2012   
1  EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...             2012   
2  (unit), EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMP...             2012   
3  EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPT...             2012   
4  Provinces  Autonomous , Number of, Cities, Num...             2012   

          section   group  row_id is_title is_empty  original_index     label  
0  General Survey  B0101e       1      yes       no               0     title  
1  General Survey  B0101e       2       no      yes         